In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp
import time

# Testing Gurobi install with basic MIP

In [3]:
# Create a new model
m = gp.Model("matrix1")

# Create variables
x = m.addMVar(shape=3, vtype=GRB.BINARY, name="x")

# Set objective
obj = np.array([1.0, 1.0, 2.0])
m.setObjective(obj @ x, GRB.MAXIMIZE)

# Build (sparse) constraint matrix
val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
row = np.array([0, 0, 0, 1, 1])
col = np.array([0, 1, 2, 0, 1])

A = sp.csr_matrix((val, (row, col)), shape=(2, 3))

# Build rhs vector
rhs = np.array([4.0, -1.0])

# Add constraints
m.addConstr(A @ x <= rhs, name="c")

# Optimize model
m.optimize()

print(x.X)
print(f"Obj: {m.ObjVal:g}")


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 23.6.0 23H311)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x8d4960d3
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 11 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
[1. 0. 1.]
Obj: 3


# Two-way marginals

## Generate table and compute marginals

In [6]:
def generate_random_binary_table(n, d):
    """
    Generates an n x d binary table with entries 0 or 1 chosen uniformly at random.
    
    Parameters:
        n (int): Number of rows.
        d (int): Number of columns.
        
    Returns:
        np.ndarray: A binary matrix of shape (n, d).
    """
    # np.random.randint generates integers in [0,2) i.e. 0 or 1.
    table = np.random.randint(0, 2, size=(n, d))
    return table

def compute_two_way_marginals(table):
    """
    Computes the 2-way marginals for a given binary table.
    
    For every pair of column indices (i, j) with i < j, it counts the number of rows 
    that have a 1 in both column i and column j.
    
    Parameters:
        table (np.ndarray): A binary matrix of shape (n, d).
        
    Returns:
        dict: A dictionary with keys as tuples (i, j) and values as the counts.
    """
    # Compute the d x d matrix of inner products between columns.
    # The (i, j)-th entry of counts is the number of rows where both column i and column j are 1.
    counts = table.T @ table
    
    # Use np.triu_indices to extract indices where i < j.
    d = table.shape[1]
    i_indices, j_indices = np.triu_indices(d, k=0)
    
    # Create the dictionary of 2-way monotone marginals.
    # Includes 1-way marginals since it includes cases where i ==j
    marginals = {(i, j): counts[i, j] for i, j in zip(i_indices, j_indices)}
    
    
    return marginals


In [7]:
n1 = 3
d1 = 5
table1 = generate_random_binary_table(n1, d1)
marginals1 = compute_two_way_marginals(table1)

In [8]:
print(marginals1)

{(0, 0): 1, (0, 1): 0, (0, 2): 1, (0, 3): 1, (0, 4): 0, (1, 1): 1, (1, 2): 0, (1, 3): 0, (1, 4): 1, (2, 2): 2, (2, 3): 2, (2, 4): 1, (3, 3): 2, (3, 4): 1, (4, 4): 2}


## Running Gurobi solver

In [42]:
def solve_table_from_marginals(n, d, marginals, verbose = False):
    """
    Given the number of rows n, the number of columns d, and a dictionary of 2-way marginals,
    this function builds and solves a Gurobi model to find a binary table that is consistent with
    the provided marginals.

    Parameters:
        n (int): Number of rows in the table.
        d (int): Number of columns in the table.
        marginals (dict): A dictionary where keys are tuples (i, j) with 0 <= i < j < d and 
                          values are the counts of rows with a 1 in both columns i and j.

    Returns:
        list of lists: A 2D list representing the binary table, or None if no solution exists.
        Gurobi model: the actual Gurobi object (for inspection)
    """
    # Create a new model.
    m = gp.Model("table_from_marginals")
    
    # Turn off Gurobi output if desired (set to 1 to see details)
    if verbose: 
        m.Params.OutputFlag = 1
    else:
        m.Params.OutputFlag = 0

    # Create binary variables x[r,c] for each row r and column c.
    x = m.addVars(n, d, vtype=GRB.BINARY, name="x")

    # For every pair of columns (i, j) in the marginals dictionary,
    # add a quadratic constraint that forces the inner product of column i and column j
    # to equal the provided marginal value.
    for i in range(d):
        for j in range(i + 1, d):
            m.addQConstr(
                ########################################
                ########################################
                # ADD THE ACTUAL CONSTRAINT HERE!
                0,
                ########################################
                ########################################
                name=f"marginal_{i}_{j}")
    
    m.setObjective(0,
                   GRB.MINIMIZE)
    m.Params.PoolSolutions = 1
    m.Params.SolutionLimit = 1

    # Optimize the model.
    m.optimize()

    # Check if a solution was found.
    if m.solCount > 0: # m.status in (GRB.OPTIMAL, GRB.FEASIBLE):
        # Extract and return the table as a list of lists.
        solution = [[int(x[r, c].X) for c in range(d)] for r in range(n)]
        return solution, m
    else:
        print("No solution was found")
        return None, m


### Let's try to reconstruct!

In [60]:
n1 = 2
d1 = 7
table1 = generate_random_binary_table(n1, d1)
marginals1 = compute_two_way_marginals(table1)
sol1, m1 = solve_table_from_marginals(n1, d1, marginals1)

print("Original:")
print(table1)
print("Reconstructed")
print(np.array(sol1))

Original:
[[0 1 1 1 1 0 0]
 [1 1 0 0 1 1 0]]
Reconstructed
[[1 1 0 0 1 1 0]
 [0 1 1 1 1 0 0]]
